https://adventofcode.com/2022/day/13

In [1]:
import re
import itertools as it
from typing import Optional
from functools import cmp_to_key

Packet = list[int, list["Packet"]]
Pair = tuple[Packet, Packet]


def parsePackets(input_file: str) -> list[Pair]:
    with open(input_file) as f:
        text = f.read()

    packets = list()
    for pair in text.split("\n\n"):
        groups = re.search(r"([\[\],\d]+)\s+([\[\],\d]+)", pair).groups()
        lr = tuple(map(lambda x: eval(x.strip()), groups))
        packets.append(lr)

    return packets


def isRightOrder(left: Packet, right: Packet) -> Optional[bool]:
    for l, r in it.zip_longest(left, right):
        tl, tr = type(l), type(r)
        result = None
        if (l is None) and (r is not None):
            result = True
        elif (l is not None) and (r is None):
            result = False
        elif all([t is int for t in (tl, tr)]):
            if l < r:
                result = True
            elif l > r:
                result = False
        elif all([t is list for t in (tl, tr)]):
            result = isRightOrder(l, r)
        elif tl is int and tr is list:
            result = isRightOrder([l], r)
        elif tl is list and tr is int:
            result = isRightOrder(l, [r])
        else:
            assert False, f"Unhandled case: {tl=}, {tr=}"

        if result is not None:
            return result

    return None


def solvePart1(pairs: list[Pair]) -> int:
    indices = [i for i, p in enumerate(pairs, 1) if isRightOrder(*p)]
    return sum(indices)


def solvePart2(pairs: list[Pair]) -> int:
    START, END = [[2]], [[6]]
    packets = [START, END]
    for p in pairs:
        packets.extend(p)

    cmp = lambda l, r: -1 if isRightOrder(l, r) else (1 if isRightOrder(l, r) is not None else 0)
    packets.sort(key=cmp_to_key(cmp))

    return (packets.index(START) + 1) * (packets.index(END) + 1)


In [2]:
packets = parsePackets("test_input.txt")

result, expected = solvePart1(packets), 13
assert result == expected, f"Part 1: {result=} is wrong ({expected=})"

result, expected = solvePart2(packets), 140
assert result == expected, f"Part 2: {result=} is wrong ({expected=})"


In [3]:
packets = parsePackets("input.txt")

print(solvePart1(packets))
print(solvePart2(packets))

6428
22464
